In [ ]:
!pip install -qq transformers
!pip install -qq optuna
!pip install -qq sentencepiece
!pip install -qq datasets

     |████████████████████████████████| 3.5 MB 14.4 MB/s 
     |████████████████████████████████| 895 kB 53.3 MB/s 
     |████████████████████████████████| 596 kB 57.9 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 6.8 MB 75.5 MB/s 
     |████████████████████████████████| 308 kB 13.2 MB/s 
     |████████████████████████████████| 210 kB 69.4 MB/s 
     |████████████████████████████████| 80 kB 10.8 MB/s 
     |████████████████████████████████| 75 kB 5.0 MB/s 
     |████████████████████████████████| 149 kB 87.0 MB/s 
     |████████████████████████████████| 49 kB 7.8 MB/s 
     |████████████████████████████████| 113 kB 83.6 MB/s 
     |████████████████████████████████| 1.2 MB 14.4 MB/s 
     |████████████████████████████████| 311 kB 14.1 MB/s 
     |████████████████████████████████| 243 kB 71.4 MB/s 
     |████████████████████████████████| 1.1 MB 63.4 MB/s 
     |████████████████████████████████| 133 kB 82.5 MB/s 
     |███████████████

In [ ]:
import transformers
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AdamW, get_linear_schedule_with_warmup,Trainer, TrainingArguments
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
import random
from textwrap import wrap
from datetime import datetime
from datasets import load_from_disk
from datasets import Dataset
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
MODEL_NAME = "distilbert-base-uncased"
# max sequence length for each document/sentence sample
MAX_LENGTH = 64
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE= 1e-5
WEIGHT_DECAY = 1e-16
RANDOM_SEED=5

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
def set_seed(seed):
    """Set all seeds to make results reproducible (deterministic mode).
       When seed is None, disables deterministic mode.
    :param seed: an integer to your choosing
    """
    if seed is not None:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        np.random.seed(seed)
        random.seed(seed)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)
  confusion_matrix = classification_report(labels, preds, digits=4,output_dict=True)
  return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'hate_f1': confusion_matrix["0"]["f1-score"],
        'hate_recall': confusion_matrix["0"]["recall"],
        'hate_precision': confusion_matrix["0"]["precision"],
        'offensive_f1': confusion_matrix["1"]["f1-score"],
        'offensive_recall': confusion_matrix["1"]["recall"],
        'offensive_precision': confusion_matrix["1"]["precision"],
        'normal_f1': confusion_matrix["2"]["f1-score"],
        'normal_recall': confusion_matrix["2"]["recall"],
        'normal_precision': confusion_matrix["2"]["precision"],    
  }

def model_init():
  temp_model =  AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels=3).to(device)
  return temp_model

def timestamp():
    dateTimeObj = datetime.now()
    timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S.%f)")
    print(timestampStr)

In [ ]:
set_seed(RANDOM_SEED)




In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
dataset_dfs = load_from_disk('/content/drive/MyDrive/Dissertation/datasets/hatetwit_'+str(1))


In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Dissertation/disbert_hate_twit/hyper/results',          # output directory
    num_train_epochs=EPOCHS,              # total number of training epochs
    save_strategy ="epoch" ,
    per_device_train_batch_size=BATCH_SIZE,  # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,   # batch size for evaluation
    weight_decay= WEIGHT_DECAY,               # strength of weight decay
    learning_rate= LEARNING_RATE, 
    logging_dir='./disbert_hate/hyper/logs',     # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    evaluation_strategy="epoch",
    #eval_steps = 500     # evaluate each `logging_steps`
)

Each argument is explained in the code comments, I've specified 16 as training batch size, that's because it's the maximum I can get to fit in a Google Colab environment's memory.

You can also tweak other parameters, such as increasing the number of epochs for better training.

***Note that load best model at end set to False as issue with saving steps value

In [ ]:
hyper_trainer = Trainer(
    model_init=model_init,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset_dfs['train'],         # training dataset
    eval_dataset=dataset_dfs['validation'],          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}

https://hu

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
creating metadata file for /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bia

In [ ]:
def hp_space_optuna(trial) :
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 5),
        "seed": trial.suggest_int("seed", 1, 40),
        "warmup_steps": trial.suggest_int("warmup_steps", 0, 500),
        "weight_decay": trial.suggest_float("weight_decay", 0, 0.3),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [ 8, 16, 32, 64]),
    }

In [ ]:
best_trial = hyper_trainer.hyperparameter_search(n_trials=40, direction="maximize", backend="optuna", hp_space=hp_space_optuna)

[I 2022-02-12 13:32:41,386] A new study created in memory with name: no-name-60724ab9-d73e-4387-8c41-db9ac9239eee
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinu

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.655000,0.595284,0.761322,0.757030,0.756396,0.761322
2,0.574900,0.561695,0.772269,0.770109,0.770437,0.772269
3,0.544600,0.552900,0.775274,0.771375,0.770792,0.775274


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: input_ids_bert, sentence, token_type_ids_bert, __index_level_0__, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16


              precision    recall  f1-score   support

           0     0.6783    0.7254    0.7010       994
           1     0.8235    0.8601    0.8414      2701
           2     0.6490    0.5218    0.5785       964

    accuracy                         0.7613      4659
   macro avg     0.7169    0.7024    0.7070      4659
weighted avg     0.7564    0.7613    0.7570      4659

0.7570295544860335


Saving model checkpoint to /content/drive/MyDrive/Dissertation/disbert_hate_twit/hyper/results/run-0/checkpoint-1165
Configuration saved in /content/drive/MyDrive/Dissertation/disbert_hate_twit/hyper/results/run-0/checkpoint-1165/config.json
Model weights saved in /content/drive/MyDrive/Dissertation/disbert_hate_twit/hyper/results/run-0/checkpoint-1165/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: input_ids_bert, sentence, token_type_ids_bert, __index_level_0__, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16


...

[I 2022-02-12 15:47:32,568] Trial 38 pruned. 
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_ve

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.606900,0.571793,0.776132,0.772837,0.771481,0.776132


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: input_ids_bert, sentence, token_type_ids_bert, __index_level_0__, attention_mask_bert.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16


              precision    recall  f1-score   support

           0     0.7067    0.7103    0.7085       994
           1     0.8332    0.8726    0.8524      2701
           2     0.6655    0.5737    0.6162       964

    accuracy                         0.7761      4659
   macro avg     0.7351    0.7189    0.7257      4659
weighted avg     0.7715    0.7761    0.7728      4659

0.7728368609924559


[I 2022-02-12 15:50:17,389] Trial 39 pruned. 


In [ ]:
timestamp()

12-Feb-2022 (15:50:17.397670)


In [ ]:
best_trial

BestRun(run_id='9', objective=3.2486894835841547, hyperparameters={'learning_rate': 6.579941859647633e-05, 'num_train_epochs': 3, 'seed': 22, 'warmup_steps': 464, 'weight_decay': 0.2894897525763174, 'per_device_train_batch_size': 16})